In [ ]:
#| default_exp drr

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations

import numpy as np
import torch
import torch.nn as nn
from fastcore.basics import patch

from diffdrr.detector import Detector
from diffdrr.renderers import Siddon, Trilinear

## DRR
`DRR` is a PyTorch module that compues differentiable digitally reconstructed radiographs. The viewing angle for the DRR (known generally in computer graphics as the *camera pose*) is parameterized by the following parameters:

- SDD : source-to-detector distance (i.e., the focal length of the C-arm)
- $\mathbf R \in \mathrm{SO}(3)$ : a rotation
- $\mathbf t \in \mathbb R^3$ : a translation

If using Euler angles, the parameters are

- `alpha`      : Azimuthal angle
- `beta`       : Polar angle
- `gamma`      : Plane rotation angle
- `bx`         : X-dir translation
- `by`         : Y-dir translation
- `bz`         : Z-dir translation
- `convention` : Order of angles (e.g., `ZYX`)

`(bx, by, bz)` are translational parameters and `(alpha, beta, gamma)` are rotational parameters.

In [ ]:
#| export
from torchio import Subject

from diffdrr.pose import RigidTransform


class DRR(nn.Module):
    """PyTorch module that computes differentiable digitally reconstructed radiographs."""

    def __init__(
        self,
        subject: Subject,  # TorchIO wrapper for the CT volume
        sdd: float,  # Source-to-detector distance (i.e., the C-arm's focal length)
        height: int,  # Height of the rendered DRR
        delx: float,  # X-axis pixel size
        width: int | None = None,  # Width of the rendered DRR (default to `height`)
        dely: float | None = None,  # Y-axis pixel size (if not provided, set to `delx`)
        x0: float = 0.0,  # Principal point X-offset
        y0: float = 0.0,  # Principal point Y-offset
        p_subsample: float | None = None,  # Proportion of pixels to randomly subsample
        reshape: bool = True,  # Return DRR with shape (b, 1, h, w)
        reverse_x_axis: bool = True,  # If True, obey radiologic convention (e.g., heart on right)
        patch_size: int | None = None,  # Render patches of the DRR in series
        renderer: str = "siddon",  # Rendering backend, either "siddon" or "trilinear"
        persistent: bool = True,  # Set persistent value in `torch.nn.Module.register_buffer`
        **renderer_kwargs,  # Kwargs for the renderer
    ):
        super().__init__()

        # Initialize the X-ray detector
        width = height if width is None else width
        dely = delx if dely is None else dely
        if p_subsample is not None:
            n_subsample = int(height * width * p_subsample)
        else:
            n_subsample = None
        self.detector = Detector(
            sdd,
            height,
            width,
            delx,
            dely,
            x0,
            y0,
            subject.reorient,
            reverse_x_axis=reverse_x_axis,
            n_subsample=n_subsample,
        )

        # Initialize the volume and world geometry
        self.subject = subject
        self.volume = subject.volume.data.squeeze()
        self.register_buffer(
            "density",
            subject.density.data.squeeze(),
            persistent=persistent,
        )
        self.register_buffer(
            "_affine",
            torch.as_tensor(subject.volume.affine, dtype=torch.float32).unsqueeze(0),
            persistent=persistent,
        )  # Using float64 can sometimes improve rendering quality (https://github.com/eigenvivek/DiffDRR/issues/202)
        self.register_buffer(
            "_affine_inverse",
            self._affine.inverse(),
            persistent=persistent,
        )
        self.register_buffer(
            "density",
            subject.density.data.squeeze(),
            persistent=persistent,
        )
        if subject.mask is not None:
            self.register_buffer(
                "mask",
                subject.mask.data.to(torch.float32).squeeze(),
                persistent=persistent,
            )

        # Initialize the renderer
        if renderer == "siddon":
            self.renderer = Siddon(**renderer_kwargs)
        elif renderer == "trilinear":
            self.renderer = Trilinear(**renderer_kwargs)
        else:
            raise ValueError(
                f"renderer must be 'siddon' or 'trilinear', not {renderer}"
            )
        self.reshape = reshape
        self.patch_size = patch_size
        if self.patch_size is not None:
            self.n_patches = (height * width) // (self.patch_size**2)

    def reshape_transform(self, img, batch_size):
        if self.reshape:
            if self.detector.n_subsample is None:
                img = img.view(
                    batch_size, -1, self.detector.height, self.detector.width
                )
            else:
                img = reshape_subsampled_drr(img, self.detector, batch_size)
        return img

    @property
    def affine(self):
        return RigidTransform(self._affine)

    @property
    def affine_inverse(self):
        return RigidTransform(self._affine_inverse)

In [ ]:
#| exporti
def reshape_subsampled_drr(img: torch.Tensor, detector: Detector, batch_size: int):
    n_points = detector.height * detector.width
    drr = torch.zeros(batch_size, n_points).to(img)
    drr[:, detector.subsamples[-1]] = img
    drr = drr.view(batch_size, 1, detector.height, detector.width)
    return drr

The forward pass of the `DRR` module generated DRRs from the input CT volume. The pose parameters (i.e., viewing angles) from which the DRRs are generated are passed to the forward call.

In [ ]:
#| export
from diffdrr.pose import RigidTransform, convert


@patch
def forward(
    self: DRR,
    *args,  # Some batched representation of SE(3)
    parameterization: str = None,  # Specifies the representation of the rotation
    convention: str = None,  # If parameterization is Euler angles, specify convention
    calibration: RigidTransform = None,  # Optional calibration matrix with the detector's intrinsic parameters
    mask_to_channels: bool = False,  # If True, structures from the CT mask are rendered in separate channels
    **kwargs,  # Passed to the renderer
):
    """Generate DRR with rotational and translational parameters."""
    # Initialize the camera pose
    if parameterization is None:
        pose = args[0]
    else:
        pose = convert(*args, parameterization=parameterization, convention=convention)
    source, target = self.detector(pose, calibration)
    source = self.affine_inverse(source)
    target = self.affine_inverse(target)

    # Render the DRR
    kwargs["mask"] = self.mask if mask_to_channels else None
    if self.patch_size is None:
        img = self.renderer(
            self.density,
            source,
            target,
            **kwargs,
        )
    else:
        n_points = target.shape[1] // self.n_patches
        img = []
        for idx in range(self.n_patches):
            t = target[:, idx * n_points : (idx + 1) * n_points]
            partial = self.renderer(
                self.density,
                source,
                t,
                **kwargs,
            )
            img.append(partial)
        img = torch.cat(img, dim=-1)
    return self.reshape_transform(img, batch_size=len(pose))

In [ ]:
#| exporti
@patch
def set_intrinsics(
    self: DRR,
    sdd: float = None,
    delx: float = None,
    dely: float = None,
    x0: float = None,
    y0: float = None,
):
    self.detector = Detector(
        sdd if sdd is not None else self.detector.sdd,
        self.detector.height,
        self.detector.width,
        delx if delx is not None else self.detector.delx,
        dely if dely is not None else self.detector.dely,
        x0 if x0 is not None else self.detector.x0,
        y0 if y0 is not None else self.detector.y0,
        n_subsample=self.detector.n_subsample,
        reverse_x_axis=self.detector.reverse_x_axis,
        reorient=self.subject.reorient,
    ).to(self.volume)

In [ ]:
#| export
@patch
def perspective_projection(
    self: DRR,
    pose: RigidTransform,
    pts: torch.Tensor,
):
    """Project points in world coordinates (3D) onto the pixel plane (2D)."""
    extrinsic = (self.detector.reorient.compose(pose)).inverse()
    x = extrinsic(pts)
    x = torch.einsum("ij, bnj -> bni", self.detector.intrinsic, x)
    z = x[..., -1].unsqueeze(-1).clone()
    x = x / z
    if self.detector.reverse_x_axis:
        x[..., 1] = self.detector.width - x[..., 1]
    return x[..., :2].flip(-1)

In [ ]:
#| export
from torch.nn.functional import pad


@patch
def inverse_projection(
    self: DRR,
    pose: RigidTransform,
    pts: torch.Tensor,
):
    """Backproject points in pixel plane (2D) onto the image plane in world coordinates (3D)."""
    pts = pts.flip(-1)
    if self.detector.reverse_x_axis:
        pts[..., 1] = self.detector.width - pts[..., 1]
    x = self.detector.sdd * torch.einsum(
        "ij, bnj -> bni",
        self.detector.intrinsic.inverse(),
        pad(pts, (0, 1), value=1),  # Convert to homogenous coordinates
    )
    extrinsic = self.detector.reorient.compose(pose)
    return extrinsic(x)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()